In [1]:
from pyspark import SparkContext
import pyspark
import time

conf = pyspark.SparkConf().set("spark.driver.host", "localhost")
sc = SparkContext.getOrCreate()
sc.setLogLevel('OFF')

21/12/07 23:25:27 WARN Utils: Your hostname, Fans-MacBook-Air.local resolves to a loopback address: 127.0.0.1; using 10.0.0.115 instead (on interface en0)
21/12/07 23:25:27 WARN Utils: Set SPARK_LOCAL_IP if you need to bind to another address
Using Spark's default log4j profile: org/apache/spark/log4j-defaults.properties
Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).
21/12/07 23:25:28 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable


In [2]:
business_sf = sc.textFile(
    "../data/SF_business/filtered_registered_business_sf.tsv")
supervisor_sf = sc.textFile("../data/SF_business/supervisor_sf.tsv")

In [3]:
partition_ct = 4

## 1. Create an RDD which has partition_ct as the number of partitions  in a format of (zip, supervisor_id) order by zip.

In [10]:
supervisor_sf= supervisor_sf.map(lambda x: (int(x.split('\t')[0]),int(x.split('\t')[1])))

In [12]:
sorted_zip_supervisor_id = supervisor_sf.sortByKey(numPartitions=partition_ct)

## how the data are organized?
- Do same keys locate in the same partition?

In [13]:
sorted_zip_supervisor_id.glom().collect()

[[(94102, 8),
  (94102, 6),
  (94102, 3),
  (94102, 5),
  (94103, 8),
  (94103, 9),
  (94103, 10),
  (94103, 6),
  (94103, 3),
  (94103, 5),
  (94104, 6),
  (94104, 3),
  (94105, 6),
  (94105, 3),
  (94107, 10),
  (94107, 6),
  (94108, 6),
  (94108, 3),
  (94109, 2),
  (94109, 6),
  (94109, 3),
  (94109, 5)],
 [(94110, 8),
  (94110, 11),
  (94110, 9),
  (94110, 10),
  (94111, 6),
  (94111, 3),
  (94112, 7),
  (94112, 8),
  (94112, 11),
  (94112, 9),
  (94112, 10),
  (94114, 7),
  (94114, 8),
  (94114, 5),
  (94115, 2),
  (94115, 1),
  (94115, 5)],
 [(94116, 7),
  (94116, 4),
  (94117, 1),
  (94117, 7),
  (94117, 8),
  (94117, 5),
  (94118, 2),
  (94118, 1),
  (94118, 5),
  (94121, 2),
  (94121, 1),
  (94122, 1),
  (94122, 7),
  (94122, 5),
  (94122, 4),
  (94123, 2),
  (94124, 9),
  (94124, 10)],
 [(94127, 7),
  (94127, 8),
  (94127, 11),
  (94129, 2),
  (94130, 6),
  (94131, 7),
  (94131, 8),
  (94131, 5),
  (94132, 7),
  (94132, 11),
  (94132, 4),
  (94133, 2),
  (94133, 3),
  (94134

## 2. Create an RDD which has partition_ct as the number of partitions in a format of (zip, [name, street, city, zip]) partitioned by zip (hash).

In [14]:
business_sf.take(5)

['94123\tTournahu George L\t3301 Broderick St\tSan Francisco\tCA',
 '94124\tStephens Institute Inc\t2225 Jerrold Ave\tSan Francisco\tCA',
 '94105\tStephens Institute Inc\t180 New Montgomery St\tSan Francisco\tCA',
 '94108\tStephens Institute Inc\t540 Powell St\tSan Francisco\tCA',
 '94107\tStephens Institute Inc\t460 Townsend St\tSan Francisco\tCA']

In [19]:
partitioned_zip_business = business_sf.map(lambda x: x.split('\t')).filter(lambda x: len(x[0])!=0).map(lambda x: (int(x[0]),x[1:])).partitionBy(numPartitions=partition_ct)
partitioned_zip_business.collect()

[(94124,
  ['Stephens Institute Inc', '2225 Jerrold Ave', 'San Francisco', 'CA']),
 (94108, ['Stephens Institute Inc', '540 Powell St', 'San Francisco', 'CA']),
 (94108, ['Stephens Institute Inc', '740 Taylor St', 'San Francisco', 'CA']),
 (94108,
  ['"Fugazi Travel Agency, Inc."',
   '170 Grant Ave 4th Fl',
   'San Francisco',
   'CA']),
 (94132, ['Agid Hyman D', '452 Gellert Dr', 'San Francisco', 'CA']),
 (94108, ['Felton Donald & Nancy', '445 Stockton St', 'San Francisco', 'CA']),
 (94108, ['Felton Donald & Nancy', '445 Stockton St', 'San Francisco', 'CA']),
 (94124, ['Anresco Inc', '1370 Van Dyke Ave', 'San Francisco', 'CA']),
 (94124, ['Anresco Inc', '1370 Van Dyke Ave', 'San Francisco', 'CA']),
 (94108,
  ['Anthonys Shoe Service Inc', '340 Kearny St', 'San Francisco', 'CA']),
 (94112, ['Antonchuk Richard A', '1117 Ocean Ave', 'San Francisco', 'CA']),
 (94112, ['Antonchuk Richard A', '1125 Ocean Ave', 'San Francisco', 'CA']),
 (94124, ['Arnold & Egan Mfg Co', '1515 Griffith St', '

## how the data are organized?
- Do same keys locate in the same partition?

In [21]:
partitioned_zip_business.glom().take(1)

[[(94124,
   ['Stephens Institute Inc', '2225 Jerrold Ave', 'San Francisco', 'CA']),
  (94108, ['Stephens Institute Inc', '540 Powell St', 'San Francisco', 'CA']),
  (94108, ['Stephens Institute Inc', '740 Taylor St', 'San Francisco', 'CA']),
  (94108,
   ['"Fugazi Travel Agency, Inc."',
    '170 Grant Ave 4th Fl',
    'San Francisco',
    'CA']),
  (94132, ['Agid Hyman D', '452 Gellert Dr', 'San Francisco', 'CA']),
  (94108, ['Felton Donald & Nancy', '445 Stockton St', 'San Francisco', 'CA']),
  (94108, ['Felton Donald & Nancy', '445 Stockton St', 'San Francisco', 'CA']),
  (94124, ['Anresco Inc', '1370 Van Dyke Ave', 'San Francisco', 'CA']),
  (94124, ['Anresco Inc', '1370 Van Dyke Ave', 'San Francisco', 'CA']),
  (94108,
   ['Anthonys Shoe Service Inc', '340 Kearny St', 'San Francisco', 'CA']),
  (94112, ['Antonchuk Richard A', '1117 Ocean Ave', 'San Francisco', 'CA']),
  (94112, ['Antonchuk Richard A', '1125 Ocean Ave', 'San Francisco', 'CA']),
  (94124, ['Arnold & Egan Mfg Co', '1

## 3. Join sorted_zip_supervisor_id and partitioned_zip_business efficiently for any sorted_zip_supervisor_id. (assume that the joined RDDs will be repeatedly used)

In [25]:
sorted_zip_supervisor_id.cache()
partitioned_zip_business.cache()
start = time.time()
joined_supervisor_business = sorted_zip_supervisor_id.leftOuterJoin(partitioned_zip_business)
#joined_supervisor_business.cache()
joined_supervisor_business.collect()
print(time.time() - start)

3.1539840698242188


In [23]:
start = time.time()
joined_supervisor_business = sorted_zip_supervisor_id.leftOuterJoin(partitioned_zip_business)
joined_supervisor_business.cache()
joined_supervisor_business.collect()
print(time.time() - start)

2.8824331760406494


In [26]:
strat = time.time()
joined_supervisor_business.collect()
print(time.time()-strat)

1.9549686908721924


# Custom Partitioner

## 1) For the range of supervisor ids, create RDD of (supervisor_id, [business name, street, city, state]) within the particion_ct based on the supervisor_id serially.
- Ex. if partition_ct is 3, and supervisor ids are [1, 2, 3, 4, 5, 6, 7, 8, 9] => first partition should have pairs with supervisor ids of [1,2,3], second paritition should have pairs with supervisor ids of [4,5,6], third partition should have pairs with supervisor ids of [7,8,9]

In [27]:
supervisor_business = joined_supervisor_business.values()

In [31]:
supervisor_business.take(5)

[(6, ['Avenal Land & Oil Co', '57 Post St 809', 'San Francisco', 'CA']),
 (6, ['Kokjer R Etal', '333 Pine Street 5th Floor', 'San Francisco', 'CA']),
 (6,
  ['Benson & Neff-Cpas-A Prof Corp', '1 Post St 2150', 'San Francisco', 'CA']),
 (6,
  ['Berlitz School Of Languages',
   '180 Montgomery St 1580',
   'San Francisco',
   'CA']),
 (6, ['J R Bisho Company Inc', '582 Market St 1905', 'San Francisco', 'CA'])]

In [28]:
supervisor_ids = joined_supervisor_business\
    .values()\
    .keys()\
    .distinct()\
    .sortBy(lambda x: x)\
    .collect()

In [29]:
supervisor_ids

[1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11]

In [37]:
def custom_range_partition(key):
    size = (max(supervisor_ids)-min(supervisor_ids)+1)/partition_ct
    return int((key-min(supervisor_ids))/size)

In [39]:
supervisor_business_custom_range = supervisor_business.partitionBy(numPartitions=partition_ct,
                                                                   partitionFunc= custom_range_partition)
supervisor_business_custom_range.saveAsTextFile('supervisor_business_custom_range')

## 2)  Store pairs of (supervisor_id, [business name, street, city, state]) within 2 partitions, based on whether the supervisor_id is even or odd.

In [43]:
def odd_even_partitioner(key):
    if key%2 == 0:
        return 1
    else:
        return 2

In [42]:
supervisor_business.take(5)

[(6, ['Avenal Land & Oil Co', '57 Post St 809', 'San Francisco', 'CA']),
 (6, ['Kokjer R Etal', '333 Pine Street 5th Floor', 'San Francisco', 'CA']),
 (6,
  ['Benson & Neff-Cpas-A Prof Corp', '1 Post St 2150', 'San Francisco', 'CA']),
 (6,
  ['Berlitz School Of Languages',
   '180 Montgomery St 1580',
   'San Francisco',
   'CA']),
 (6, ['J R Bisho Company Inc', '582 Market St 1905', 'San Francisco', 'CA'])]

In [46]:
supervisor_business_odd_even = supervisor_business.partitionBy(numPartitions=2,partitionFunc=odd_even_partitioner)
supervisor_business_odd_even.saveAsTextFile('supervisor_business_odd_even')

In [ ]:
sc.stop()